In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# 1. Setup the Environment

Transformers: For loading and fine-tuning the pre-trained RoBERTa model.

Datasets: For loading the MasakhaNER dataset.

Seqeval: For evaluating NER tasks (accuracy, precision, recall, F1-score).

In [ ]:
pip install transformers datasets seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.0 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=74c8e345ac758713f6c69f26e462a16cb961defc7ea8f65f403f1c77dc1a6a93
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


# Load the Dataset
Load the MasakhaNER dataset, which contains Swahili text annotated with named entities.

In [ ]:
from datasets import load_dataset

# Load the MasakhaNER dataset
dataset = load_dataset("masakhaner", "swa")
print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

masakhaner.py:   0%|          | 0.00/7.60k [00:00<?, ?B/s]

The repository for masakhaner contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/masakhaner.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/2109 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/300 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/604 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2109
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 300
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 604
    })
})


In [ ]:
# Print the first five elements separately
for i in range(10):
    print(dataset["train"][i])

{'id': '0', 'tokens': ['Wizara', 'ya', 'afya', 'ya', 'Tanzania', 'imeripoti', 'Jumatatu', 'kuwa', ',', 'watu', 'takriban', '14', 'zaidi', 'wamepata', 'maambukizi', 'ya', 'Covid', '-', '19', '.'], 'ner_tags': [3, 4, 4, 4, 4, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
{'id': '1', 'tokens': ['Walioambukizwa', 'wote', 'ni', 'raia', 'wa', 'Tanzania', ',', '13', 'wakiwa', 'Dar', '-', 'es', '-', 'salaam', 'na', 'mmoja', 'mjini', 'Arusha', '.'], 'ner_tags': [0, 0, 0, 0, 0, 5, 0, 0, 0, 5, 6, 6, 6, 6, 0, 0, 0, 5, 0]}
{'id': '2', 'tokens': ['Wizara', 'ya', 'afya', 'imeripoti', 'kwamba', 'juhudi', 'za', 'kufuatilia', 'watu', 'waliokuwa', 'karibu', 'na', 'wagonjwa', 'zinaendelea', '.'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
{'id': '3', 'tokens': ['Wakati', 'wa', 'maadimisho', 'ya', 'pasaka', ',', 'wakristo', 'walikusanyika', 'kanisani', 'kwa', 'maombi', 'bila', 'kuzingatia', 'ushauri', 'wa', 'wataalam', 'wa', 'afya', '.'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# Preprocess the Data
Convert the dataset into a format suitable for the castorini/afriberta_large model. This involves tokenizing the text and aligning the labels with the tokenized input.
Explanation of add_prefix_space=True
Why is it needed? castorini/afriberta_large tokenizers treat spaces as part of the tokenization process. When working with pretokenized inputs (e.g., a list of words), the tokenizer needs to know that each word is prefixed with a space (except the first word). Setting add_prefix_space=True ensures that the tokenizer correctly handles this.

When to use it? Always use add_prefix_space=True when your input is already split into words or tokens (e.g., examples MasakhaNER dataset).

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `Swahili` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Swahili`


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification, Trainer
from datasets import Dataset

# Initialize the tokenizer outside the function
tokenizer = AutoTokenizer.from_pretrained("lelapa/InkubaLM-0.4B", trust_remote_code=True)

# Set the padding token to be the eos_token
tokenizer.pad_token = tokenizer.eos_token

def tokenize_and_align_labels(examples, tokenizer, max_length=128):
    """
    Tokenize the dataset and align labels with tokenized input.
    Preserve the original 'tokens' field for later use.
    """
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        padding="max_length",  # Ensure max length padding
        max_length=max_length,
        is_split_into_words=True
    )

    # Align labels with tokenized input
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Special token (e.g., [CLS], [SEP])
            elif word_idx != previous_word_idx:
                if word_idx < len(label):  # Ensure we don't exceed the number of labels
                    label_ids.append(label[word_idx])  # New word
                else:
                    label_ids.append(-100)  # If there are more tokens than labels, pad with -100
            else:
                label_ids.append(-100)  # Subword of the same word
            previous_word_idx = word_idx

        # Ensure label_ids are truncated or padded to match the tokenized sequence length
        label_ids = label_ids[:len(tokenized_inputs["input_ids"][i])]  # Truncate to the number of tokens
        label_ids = label_ids + [-100] * (max_length - len(label_ids))  # Pad with -100 if needed
        labels.append(label_ids)

    # Add the original 'tokens' field to the tokenized dataset
    tokenized_inputs["tokens"] = examples["tokens"]
    tokenized_inputs["labels"] = labels

    return tokenized_inputs

# Apply tokenization to the dataset
tokenized_datasets = dataset.map(lambda x: tokenize_and_align_labels(x, tokenizer), batched=True)

# Now, tokenized_datasets will be ready for use in training


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/960 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/991k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.95M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Map:   0%|          | 0/2109 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/604 [00:00<?, ? examples/s]

In [ ]:
# Check the result after tokenization
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2109
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 300
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 604
    })
})


# Prepare the Dataset for PyTorch
Make sure your dataset is properly formatted for the Trainer:

In [ ]:
tokenized_datasets.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"]
)

# Model and Training Arguments

In [ ]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.8 MB/s eta 0:00:00


In [ ]:
pip install seqeval

In [ ]:
import numpy as np
from transformers import AutoModelForTokenClassification, Trainer, TrainingArguments, AutoTokenizer, DataCollatorForTokenClassification
import evaluate  # Importing evaluate instead of datasets

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("lelapa/InkubaLM-0.4B", trust_remote_code=True)

# Set the pad token to eos_token if it's not defined
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Define the model
model = AutoModelForTokenClassification.from_pretrained("lelapa/InkubaLM-0.4B", num_labels=9)

# Define the metric
metric = evaluate.load("seqeval")  # Using evaluate to load the seqeval metric

# Ensure that the model has a mapping for the labels
label2id = model.config.label2id  # The mapping from label string to ID
id2label = model.config.id2label  # The mapping from label ID to string

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=-1)  # Get the predicted label indices
    true_labels = labels

    # Remove padding tokens (label=-100 for padding tokens)
    true_predictions = [
        [id2label[label] for label, true_label in zip(prediction, true_label) if true_label != -100]
        for prediction, true_label in zip(predictions, true_labels)
    ]
    true_labels = [
        [id2label[label] for label in true_label if label != -100] for true_label in true_labels
    ]

    return metric.compute(predictions=true_predictions, references=true_labels)

# Define the data collator
data_collator = DataCollatorForTokenClassification(tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",               # Save directory
    evaluation_strategy="epoch",          # Evaluate after each epoch
    save_strategy="epoch",                # Save model after each epoch
    logging_dir="./logs",                 # Directory for logs
    logging_steps=100,                    # Log every 100 steps
    per_device_train_batch_size=1,        # Batch size for training
    per_device_eval_batch_size=2,         # Batch size for evaluation
    num_train_epochs=4,                   # Total number of epochs (recommend more than 1)
    learning_rate=5e-5,                   # Learning rate
    weight_decay=0.01,                    # Weight decay for regularization
    load_best_model_at_end=True,          # Load the best model at the end
    metric_for_best_model="eval_overall_f1",  # Use eval_overall_f1 for selecting the best model
    warmup_steps=500,                     # Number of warmup steps
    logging_first_step=True,              # Log the first step
)

# Initialize the Trainer
trainer = Trainer(
    model=model,                           # the model to train
    args=training_args,                    # training arguments
    data_collator=data_collator,           # data collator
    train_dataset=tokenized_datasets["train"],  # training dataset
    eval_dataset=tokenized_datasets["validation"],  # evaluation dataset
    compute_metrics=compute_metrics,       # custom metrics (precision, recall, F1)
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate(tokenized_datasets["test"])
print(results)


config.json:   0%|          | 0.00/763 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.66G [00:00<?, ?B/s]

Some weights of LlamaForTokenClassification were not initialized from the model checkpoint at lelapa/InkubaLM-0.4B and are newly initialized: ['score.bias', 'score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vikalpsri090102 (vikalpsri090102-iit-madras-alumni-association) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Abel 0,Abel 1,Abel 2,Abel 3,Abel 4,Abel 5,Abel 6,Abel 7,Abel 8,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.204100,0.200334,"{'precision': 0.7268518518518519, 'recall': 0.6993318485523385, 'f1': 0.7128263337116912, 'number': 898}","{'precision': 0.8591549295774648, 'recall': 0.5951219512195122, 'f1': 0.7031700288184437, 'number': 205}","{'precision': 0.7317073170731707, 'recall': 0.6521739130434783, 'f1': 0.6896551724137931, 'number': 92}","{'precision': 0.75, 'recall': 0.4883720930232558, 'f1': 0.5915492957746479, 'number': 86}","{'precision': 0.1951219512195122, 'recall': 0.2222222222222222, 'f1': 0.2077922077922078, 'number': 36}","{'precision': 0.9556451612903226, 'recall': 0.7821782178217822, 'f1': 0.8602540834845734, 'number': 303}","{'precision': 0.8048780487804879, 'recall': 0.7857142857142857, 'f1': 0.7951807228915663, 'number': 42}","{'precision': 0.7319587628865979, 'recall': 0.8875, 'f1': 0.8022598870056498, 'number': 80}","{'precision': 0.17073170731707318, 'recall': 0.3333333333333333, 'f1': 0.22580645161290325, 'number': 21}",0.749380,0.685196,0.715852,0.947955
2,0.129600,0.221205,"{'precision': 0.7871853546910755, 'recall': 0.7661469933184856, 'f1': 0.7765237020316027, 'number': 898}","{'precision': 0.7950310559006211, 'recall': 0.624390243902439, 'f1': 0.6994535519125683, 'number': 205}","{'precision': 0.6190476190476191, 'recall': 0.7065217391304348, 'f1': 0.6598984771573604, 'number': 92}","{'precision': 0.6666666666666666, 'recall': 0.6046511627906976, 'f1': 0.6341463414634145, 'number': 86}","{'precision': 0.4117647058823529, 'recall': 0.3888888888888889, 'f1': 0.39999999999999997, 'number': 36}","{'precision': 0.8951048951048951, 'recall': 0.8448844884488449, 'f1': 0.8692699490662139, 'number': 303}","{'precision': 0.625, 'recall': 0.7142857142857143, 'f1': 0.6666666666666666, 'number': 42}","{'precision': 0.88, 'recall': 0.825, 'f1': 0.8516129032258064, 'number': 80}","{'precision': 0.7142857142857143, 'recall': 0.7142857142857143, 'f1': 0.7142857142857143, 'number': 21}",0.781213,0.745320,0.762845,0.954289
3,0.076600,0.256712,"{'precision': 0.8138006571741512, 'recall': 0.8273942093541202, 'f1': 0.8205411374930978, 'number': 898}","{'precision': 0.8141025641025641, 'recall': 0.6195121951219512, 'f1': 0.7036011080332409, 'number': 205}","{'precision': 0.8333333333333334, 'recall': 0.6521739130434783, 'f1': 0.7317073170731708, 'number': 92}","{'precision': 0.6455696202531646, 'recall': 0.5930232558139535, 'f1': 0.6181818181818183, 'number': 86}","{'precision': 0.34146341463414637, 'recall': 0.3888888888888889, 'f1': 0.36363636363636365, 'number': 36}","{'precision': 0.8057971014492754, 'recall': 0.9174917491749175, 'f1': 0.8580246913580247, 'number': 303}","{'precision': 0.7857142857142857, 'recall': 0.7857142857142857, 'f1': 0.7857142857142857, 'number': 42}","{'precision': 0.8604651162790697, 'recall': 0.925, 'f1': 0.891566265060241, 'number': 80}","{'precision': 0.8, 'recall': 0.7619047619047619, 'f1': 0.7804878048780488, 'number': 21}",0.795895,0.791832,0.793858,0.956354
4,0.055300,0.286240,"{'precision': 0.7947019867549668, 'recall': 0.8017817371937639, 'f1': 0.7982261640798225, 'number': 898}","{'precision': 0.8313253012048193, 'recall': 0.6731707317073171, 'f1': 0.7439353099730458, 'number': 205}","{'precision': 0.7741935483870968, 'recall': 0.782608695652174, 'f1': 0.7783783783783783, 'number': 92}","{'precision': 0.6666666666666666, 'recall': 0.5813953488372093, 'f1': 0.6211180124223603, 'number': 86}","{'precision': 0.35555555555555557, 'recall': 0.4444444444444444, 'f1': 0.3950617283950617, 'number': 36}","{'precision': 0.8476190476190476, 'recall': 0.8811881188118812, 'f1': 0.8640776699029127, 'number': 303}","{'precision': 0.8292682926829268, 'recall': 0.8095238095238095, 'f1': 0.8192771084337348, 'number': 42}","{'precision': 0.7951807228915663, 'recall': 0.825, 'f1': 0.8098159509202455, 'number': 80}","{'precision': 0.3125, 'recall': 0

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LABEL_0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LABEL_5 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LABEL_6 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LABEL_1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LABEL_7 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LABEL_0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LABEL_1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LABEL_2 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LABEL_7 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LABEL_8 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_

{'eval_loss': 0.2539805769920349, 'eval_ABEL_0': {'precision': 0.7664884135472371, 'recall': 0.7827669902912622, 'f1': 0.7745421795256681, 'number': 1648}, 'eval_ABEL_1': {'precision': 0.7666666666666667, 'recall': 0.6216216216216216, 'f1': 0.6865671641791045, 'number': 333}, 'eval_ABEL_2': {'precision': 0.8560606060606061, 'recall': 0.6975308641975309, 'f1': 0.7687074829931972, 'number': 162}, 'eval_ABEL_3': {'precision': 0.6795580110497238, 'recall': 0.5565610859728507, 'f1': 0.6119402985074628, 'number': 221}, 'eval_ABEL_4': {'precision': 0.4431818181818182, 'recall': 0.4431818181818182, 'f1': 0.44318181818181823, 'number': 88}, 'eval_ABEL_5': {'precision': 0.7472727272727273, 'recall': 0.8876889848812095, 'f1': 0.8114511352418559, 'number': 463}, 'eval_ABEL_6': {'precision': 0.8961038961038961, 'recall': 0.696969696969697, 'f1': 0.7840909090909091, 'number': 99}, 'eval_ABEL_7': {'precision': 0.8427672955974843, 'recall': 0.8322981366459627, 'f1': 0.8375, 'number': 161}, 'eval_ABEL_

# Save the Best Model
After training, save your fine-tuned model:

In [ ]:
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/tokenizer.model',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

In [ ]:
print(tokenized_datasets["test"].features)

{'id': Value(dtype='string', id=None), 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-DATE', 'I-DATE'], id=None), length=-1, id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}


# Extract 500 Named Entities
From the test set predictions, extract 500 named entities. These entities should represent a diverse sample of the categories (e.g., PERSON, LOCATION, ORGANIZATION, etc.) to ensure comprehensive validation.

In [ ]:
import re

def extract_named_entities(predictions, tokenized_test_set, tokenizer):
    entities = []
    label_list = tokenized_test_set.features["ner_tags"].feature.names

    for pred, example in zip(predictions, tokenized_test_set):
        input_ids = example["input_ids"]  # Tokenized input IDs
        tokens = tokenizer.convert_ids_to_tokens(input_ids)  # Convert token IDs back to words

        current_entity = []
        entity_type = None

        for idx, (p, input_id) in enumerate(zip(pred, input_ids)):
            # Skip special tokens
            if input_id in [tokenizer.pad_token_id, tokenizer.cls_token_id, tokenizer.sep_token_id]:
                continue

            # Ensure prediction is within valid range
            if p >= len(label_list):
                continue  # Skip invalid label indices

            label = label_list[p]
            word = tokens[idx]

            # Handle subwords (remove special characters like "▁" or "##")
            word = re.sub(r"^▁|##", "", word)  # SentencePiece uses "▁", BERT uses "##"

            if label.startswith("B-"):  # Beginning of an entity
                if current_entity:
                    entities.append((" ".join(current_entity), entity_type))
                current_entity = [word]
                entity_type = label.split("-")[-1]
            elif label.startswith("I-") and current_entity:  # Inside the same entity
                current_entity.append(word)
            else:  # Outside entity
                if current_entity:
                    entities.append((" ".join(current_entity), entity_type))
                    current_entity = []
                    entity_type = None

        # Add last entity if exists
        if current_entity:
            entities.append((" ".join(current_entity), entity_type))

    return entities  # Return all entities (do not slice here)

# Ensure predictions are calculated per batch to prevent memory issues
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])

# Convert predictions to tag indices safely
predictions = np.argmax(predictions, axis=2)

# Ensure dataset and predictions match in size
if len(predictions) != len(tokenized_datasets["test"]):
    raise ValueError("Mismatch between predictions and dataset size!")

# Extract named entities (pass tokenizer)
sample_entities = extract_named_entities(predictions, tokenized_datasets["test"], tokenizer)

# Ensure at least 500 entities are printed if available
# Debugging step
for entity, entity_type in sample_entities[:min(500, len(sample_entities))]:  # Print only available entities
    print(f"Entity: {entity}, Type: {entity_type}")


Entity: Yo weri Museveni, Type: PER
Entity: lingana, Type: PER
Entity: Museveni, Type: PER
Entity: uy, Type: LOC
Entity: Museveni, Type: PER
Entity: Museveni, Type: PER
Entity: Museveni, Type: PER
Entity: Uganda, Type: LOC
Entity: Ru gunda, Type: PER
Entity: Sam Ku, Type: PER
Entity: Uganda, Type: LOC
Entity: Uganda, Type: LOC
Entity: Uganda, Type: LOC
Entity: Museveni, Type: PER
Entity: Jumanne, Type: DATE
Entity: Uganda, Type: LOC
Entity: China, Type: LOC
Entity: amb, Type: LOC
Entity: Afrika, Type: LOC
Entity: China, Type: LOC
Entity: Museveni, Type: PER
Entity: Marekani, Type: LOC
Entity: Uganda, Type: LOC
Entity: Marekani, Type: LOC
Entity: Museveni, Type: PER
Entity: Uganda, Type: LOC
Entity: Jumanne, Type: DATE
Entity: amb, Type: LOC
Entity: Uganda, Type: LOC
Entity: Dubai, Type: LOC
Entity: K ennes Bw, Type: PER
Entity: VOA, Type: ORG
Entity: Washington DC, Type: LOC
Entity: Jumamosi, Type: DATE
Entity: Ba, Type: LOC
Entity: Iran, Type: LOC
Entity: Iraq, Type: LOC
Entity: Q ass